In [42]:
import numpy as np
import cvxpy as cp
import random

In [ ]:
random.seed(42)     # python random generator
np.random.seed(42)  # numpy random generator

In [43]:
def loads(p, q, p_min, p_max, p_plus, p_pot, q_min, q_max, q_minus, q_plus):
	tau_1 = (q_plus - q_max)/(p_max - p_plus)
	tau_2 = (q_minus - q_min)/(p_max - p_plus)

	rho_1 = q_max - tau_1*p_plus
	rho_2 = q_min - tau_2*p_plus
	point = np.array([p, q])

	# Inequality constraints for the optimization problem.
	G = np.array([[-1, 0], [1, 0], [1, 0], [0, -1], [0, 1], [-tau_1, 1], [tau_2, -1]])

	h = np.array([-p_min, p_max, p_pot, -q_min, q_max, rho_1, -rho_2])

	# Define and solve the CVXPY problem.
	x = cp.Variable(2)
	prob = cp.Problem(cp.Minimize(cp.sum_squares(x - point)), [G @ x <= h])
	prob.solve()

	p_feas = x.value[0]
	q_feas = x.value[1]
	return p_feas, q_feas, prob.status

In [44]:
ds = []

In [45]:
p_min = 0.0
p_max = 0.3
p_plus = 0.2
q_min = -0.3
q_max = 0.3
q_minus = -0.15
q_plus = 0.15


In [46]:
for _ in range(5000):
	p_pot = np.random.rand() * p_max
	p = np.random.rand()
	q = 2*np.random.rand() - 1
	p_feas, q_feas, status = loads(p, q, p_min, p_max, p_plus, p_pot, q_min, q_max, q_minus, q_plus)
	X = np.array([2.*p-1., q, 2.*(p_pot/p_max)-1., 2.*(p_max - 0.2)/0.8 - 1., 2.*(q_max - 0.2)/0.8 -1., 2.*(p_plus - 0.1)/(0.9*p_max - 0.1) -1., 2*(q_plus - 0.1)/(0.9*q_max - 0.1) - 1.])
	y = np.array([p_feas, q_feas])

	if status == 'optimal': ds.append([X, y])
	else: print("Error")

In [47]:
p_min = 0.0
p_max = 0.5
p_plus = 0.35
q_min = -0.5
q_max = 0.5
q_minus = -0.2
q_plus = 0.2

In [48]:
for _ in range(5000):
	p_pot = np.random.rand() * p_max
	p = np.random.rand()
	q = 2*np.random.rand() - 1
	p_feas, q_feas, status = loads(p, q, p_min, p_max, p_plus, p_pot, q_min, q_max, q_minus, q_plus)
	X = np.array([2.*p-1., q, 2.*(p_pot/p_max)-1., 2.*(p_max - 0.2)/0.8 - 1., 2.*(q_max - 0.2)/0.8 -1., 2.*(p_plus - 0.1)/(0.9*p_max - 0.1) -1., 2*(q_plus - 0.1)/(0.9*q_max - 0.1) - 1.])
	y = np.array([p_feas, q_feas])

	if status == 'optimal':  ds.append([X, y])
	else: print("Error") 

In [49]:
ds[0]

[array([ 0.63868119,  0.85628106,  0.94805421, -0.75      , -0.75      ,
         0.17647059, -0.41176471]),
 array([0.2, 0.3])]

In [50]:
from pickle import dump
dump(ds, open("projection.pkl", "wb"))